In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 25.9 MB/s eta 0:00:00


In [2]:
!pip install datasets==1.0.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.7 MB/s eta 0:00:00


In [3]:
import requests
import json
import torch
import os
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'):
  os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

In [5]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [6]:
from datasets import load_dataset

# Download the SQuAD dataset
dataset = load_dataset('squad')
print(dataset)
print()
# Print the first example in the training set
print(dataset['train'][0])
print()
# Access the context and question of the first example
context = dataset['train'][0]['context']
question = dataset['train'][0]['question']

# Print the context and question
print('Context:', context)
print()
print('Question:', question)


Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/955 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41. Subsequent calls will reuse this data.
DatasetDict({'train': Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599), 'validation': Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 10570)})

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}

In [7]:
train_data = dataset['train']
train_data

Dataset(features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, num_rows: 87599)

In [8]:
len(train_data)

87599

In [9]:
print(train_data[0])

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '5733be284776f41900661182', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'title': 'University_of_Notre_Dame'}


In [10]:
print(train_data[0]['context'])
print(train_data[0]['question'])
print(train_data[0]['answers']['text'][0])

Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Saint Bernadette Soubirous


In [11]:
print(train_data[99]['context'])


One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record of 105 wins, 12 losses, and five ties. During his 13 years the Irish won three national championships, had five undefeated seasons, won the Rose Bowl in 1925, and produced players such as George Gipp and the "Four Horsemen". Knute Rockne has the highest winning percentage (.881) in NCAA Division I/FBS football history. Rockne's offenses employed the Notre Dame Box and his defenses ran a 7–2–2 scheme. The last game Rockne coached was on December 14, 1930 when he led a group of Notre Dame all-stars against the New York Giants in New York City.


In [12]:
%%time
context = []
question = []
answer = []

for i in range(len(train_data)):

    context.append(train_data[i]['context'])
    question.append(train_data[i]['question'])
    answer.append(train_data[i]['answers'])

CPU times: user 13.7 s, sys: 201 ms, total: 13.9 s
Wall time: 13.8 s


In [13]:
print(len(context))
print(len(question))
print(len(answer))

87599
87599
87599


In [14]:
train_size = 2000
test_size = 500
train_contexts, train_questions, train_answers = context[0:train_size], question[0:train_size], answer[0:train_size]
valid_contexts, valid_questions, valid_answers = context[train_size:train_size+test_size], question[train_size:train_size+test_size], answer[train_size:train_size+test_size]

In [15]:
print(len(train_contexts))
print(len(train_questions))
print(len(train_answers))


2000
2000
2000


In [16]:
print(len(valid_contexts))
print(len(valid_questions))
print(len(valid_answers))

500
500
500


In [17]:
# print a random question and answer
print(f'There are {len(train_questions)} questions')
print(train_questions[-1000])
print(train_answers[-1000])

There are 2000 questions
How much did Beyonce initially contribute to the foundation?
{'answer_start': [190], 'text': ['$250,000']}


In [18]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    answer['answer_start'] = start_idx
    end_idx = start_idx + len(gold_text)
    # print(start_idx,gold_text,end_idx)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)

In [19]:
valid_answers[0]

{'answer_start': 139, 'text': ['her husband and daughter'], 'answer_end': 163}

In [20]:
# You can see that now we get the answer_end also
print(train_questions[-1000])
print(train_answers[-1000])

How much did Beyonce initially contribute to the foundation?
{'answer_start': 190, 'text': ['$250,000'], 'answer_end': 198}


In [21]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1-squad")

model = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-large-cased-v1.1-squad")

In [22]:
# from transformers import BertTokenizerFast

# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True, max_length = 512)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True, max_length = 512)

In [23]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [24]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 2000 context-question pairs


In [25]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend " venite ad me omnes ". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ), is a simple, modern stone statue of mary. [SEP] to whom did the virgin mary allegedly appear in 1858 in lourdes france? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [26]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)

In [27]:
train_encodings['start_positions'][:10]

[124, 42, 66, 91, 22, 53, 91, 118, 29, 177]

In [28]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [29]:
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

In [30]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

## Fine Tune

In [31]:
# from transformers import BertForQuestionAnswering

# model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

In [32]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [33]:
%%time
from transformers import AdamW

N_EPOCHS = 2
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 2: 100%|██████████| 125/125 [02:17<00:00,  1.10s/it, loss=0.247]

CPU times: user 3min 34s, sys: 1min 5s, total: 4min 39s
Wall time: 4min 44s


In [34]:
loss

tensor(0.2471, device='cuda:0', grad_fn=<DivBackward0>)

**Save the model in my drive in order not to run it each time**

In [35]:
#model_path = '/content/drive/MyDrive/BERT-SQuAD'
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)

**Respectively, load the saved model**

In [36]:
#from transformers import BertForQuestionAnswering, BertTokenizerFast

#model_path = '/content/drive/MyDrive/BERT-SQuAD'
#model = BertForQuestionAnswering.from_pretrained(model_path)
#tokenizer = BertTokenizerFast.from_pretrained(model_path)

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#print(f'Working on {device}')

#model = model.to(device)

### **Test**

In [37]:
model.eval()

acc = []
predict_start_index = []
predict_end_index = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    predict_start_index.append(start_pred)
    predict_end_index.append(end_pred)


100%|██████████| 32/32 [00:11<00:00,  2.88it/s]


In [49]:
def get_prediction(question,context):
    model.eval()

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length = 512)
    inputs.to(device)

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(outputs[1]) + 1 

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer

In [50]:
type(valid_contexts)

list

In [51]:
len(valid_contexts)

500

In [52]:
valid_contexts[50]

'Chopin took the new salon genre of the nocturne, invented by the Irish composer John Field, to a deeper level of sophistication. He was the first to write ballades and scherzi as individual concert pieces. He essentially established a new genre with his own set of free-standing preludes (Op. 28, published 1839). He exploited the poetic potential of the concept of the concert étude, already being developed in the 1820s and 1830s by Liszt, Clementi and Moscheles, in his two sets of studies (Op. 10 published in 1833, Op. 25 in 1837).'

In [53]:
valid_questions[50]

'What new genre di John Field invent?'

In [54]:
valid_answers[50]

{'answer_start': 39, 'text': ['nocturne'], 'answer_end': 47}

In [55]:
answer = get_prediction(valid_questions[50], valid_contexts[50])
answer

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'the nocturne'

In [56]:
original_answer = []

for i in range(len(valid_answers)):

     original_answers = valid_answers[i]['text']
     original_answer.append(original_answers)

In [57]:
predicted_answer = []

for i in range(len(valid_questions)):

  try:
     answer = get_prediction(valid_questions[i], valid_contexts[i])
     predicted_answer.append(answer)
  except:
     print(i)
     print(valid_questions[i])
     print(valid_contexts[i])

In [58]:
import pandas as pd
# Create a dictionary from the lists
data = {'Question': valid_questions, 'Context': valid_contexts, 'Original_Answer': original_answer, 'Predicted_Answer': predicted_answer}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

In [59]:
df.sample(10)

,Question,Context,Original_Answer,Predicted_Answer
120,"What two people claimed that Op 58 was ""worthy...","The two mature piano sonatas (No. 2, Op. 35, w...",[Kornel Michałowski and Jim Samson],kornel michałowski and jim samson
392,When was the Manchu Qing dynasty invasion?,"Elliot Sperling, a specialist of Indian studie...",[the 18th century],18th century
486,What's the storage capacity of the iPod Shuffle?,"Like other digital music players, iPods can se...",[2 GB],2 gb
406,What ideology was sponsored at the Ming court?,During the reign of the Jiajing Emperor (r. 15...,[the native Chinese ideology of Daoism],daoism
394,Josef Kolmaš states that Tibet became subject...,"Josef Kolmaš, a sinologist, Tibetologist, and ...",[the Chinese central government],chinese central government
390,What does Sperling claim did not have any cont...,"Elliot Sperling, a specialist of Indian studie...",[China's intervening Ming dynasty],china ' s intervening ming dynasty
456,Where did Güshi Khan resettle his tribe?,When an ally of the Ü-Tsang ruler threatened d...,[Amdo],amdo
489,Which operating systems are compatible with iT...,Apple's iTunes software (and other alternative...,[Apple Macintosh and Microsoft Windows],apple macintosh and microsoft windows
221,Which dynasty became ruler of Tibet?,"In 1358, the Sakya viceregal regime installed ...",[the Phagmodrupa Dynasty],phagmodrupa dynasty
443,Why didn't Yonten Gyatso make it to Beijing?,Chen states that the fourth Dalai Lama Yonten ...,[he died],he died


In [60]:
df.shape

(500, 4)